## Perspective correction

This issue comes from a non ideal placement of the beam with respect to the camera lens. This causes the beam to appear bigger on one side of the image. We want to correct this to have consistent mapping from pixel to real-world distance.

# 1. Canny edge detector and Hough-transforms (does not perform well for the datset)

The idea behind the script below is to detect edges and straight lines trough canny and Hough Transform respectively and classify them into right, bottom,left and upper lines. Thelongest detected lines are assumed to be the contours of the beam. The intersection of th elines are assumed to be the corners of the beam. After that a persepctive matrix is computed and the warping of the image is corrected.

In [4]:
import os
import cv2
import numpy as np

def correct_perspective(image_path):
    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error loading image {image_path}")
        return None

    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian Blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Edge detection using Canny
    edges = cv2.Canny(blurred, 50, 150, apertureSize=3)

    # Detect lines using HoughLinesP
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=100,
                            minLineLength=100, maxLineGap=10)
    if lines is None:
        print(f"No lines detected in image {image_path}")
        return None

    # Prepare to store the longest lines on each side
    left_lines = []
    right_lines = []
    top_lines = []
    bottom_lines = []

    img_center = image.shape[1] / 2  # Image center x-coordinate

    for line in lines:
        x1, y1, x2, y2 = line[0]
        # Calculate line angle
        angle = np.degrees(np.arctan2(y2 - y1, x2 - x1))

        if abs(angle) < 10:
            # Near horizontal lines
            if y1 < image.shape[0] / 2 and y2 < image.shape[0] / 2:
                top_lines.append(line[0])
            elif y1 > image.shape[0] / 2 and y2 > image.shape[0] / 2:
                bottom_lines.append(line[0])
        elif abs(angle) > 80:
            # Near vertical lines
            if x1 < img_center and x2 < img_center:
                left_lines.append(line[0])
            elif x1 > img_center and x2 > img_center:
                right_lines.append(line[0])

    if not left_lines or not right_lines or not top_lines or not bottom_lines:
        print(f"Could not find enough lines in image {image_path}")
        return None

    # Get the longest line in each group
    left_line = max(left_lines, key=lambda l: np.hypot(l[2] - l[0], l[3] - l[1]))
    right_line = max(right_lines, key=lambda l: np.hypot(l[2] - l[0], l[3] - l[1]))
    top_line = max(top_lines, key=lambda l: np.hypot(l[2] - l[0], l[3] - l[1]))
    bottom_line = max(bottom_lines, key=lambda l: np.hypot(l[2] - l[0], l[3] - l[1]))

    # Find intersection points (corners)
    def line_intersection(line1, line2):
        xdiff = (line1[0] - line1[2], line2[0] - line2[2])
        ydiff = (line1[1] - line1[3], line2[1] - line2[3])

        def det(a, b):
            return a[0] * b[1] - a[1] * b[0]

        div = det(xdiff, ydiff)
        if div == 0:
            return None  # Lines do not intersect

        d = (det((line1[0], line1[1]), (line1[2], line1[3])),
             det((line2[0], line2[1]), (line2[2], line2[3])))
        x = det(d, xdiff) / div
        y = det(d, ydiff) / div
        return [x, y]

    # Get intersection points
    top_left = line_intersection(left_line, top_line)
    top_right = line_intersection(right_line, top_line)
    bottom_left = line_intersection(left_line, bottom_line)
    bottom_right = line_intersection(right_line, bottom_line)

    if None in (top_left, top_right, bottom_left, bottom_right):
        print(f"Could not compute corner points for image {image_path}")
        return None

    # Define source and destination points
    src_pts = np.float32([top_left, top_right, bottom_right, bottom_left])

    # Compute width and height of the new image
    width_top = np.hypot(top_right[0] - top_left[0], top_right[1] - top_left[1])
    width_bottom = np.hypot(bottom_right[0] - bottom_left[0], bottom_right[1] - bottom_left[1])
    max_width = max(int(width_top), int(width_bottom))

    height_left = np.hypot(bottom_left[0] - top_left[0], bottom_left[1] - top_left[1])
    height_right = np.hypot(bottom_right[0] - top_right[0], bottom_right[1] - top_right[1])
    max_height = max(int(height_left), int(height_right))

    dst_pts = np.float32([[0, 0],
                          [max_width - 1, 0],
                          [max_width - 1, max_height - 1],
                          [0, max_height - 1]])

    # Compute the perspective transform matrix and apply it
    M = cv2.getPerspectiveTransform(src_pts, dst_pts)
    warped = cv2.warpPerspective(image, M, (max_width, max_height))

    return warped

def process_images(input_folder, output_folder):
    # Create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Get list of image files in the input folder
    image_files = [f for f in os.listdir(input_folder)
                   if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

    if not image_files:
        print(f"No images found in {input_folder}")
        return

    for image_file in image_files:
        image_path = os.path.join(input_folder, image_file)
        result = correct_perspective(image_path)

        if result is not None:
            # Save the result in the output folder
            output_path = os.path.join(output_folder, image_file)
            cv2.imwrite(output_path, result)
            print(f"Processed and saved: {output_path}")
        else:
            print(f"Failed to process image {image_path}")

def main():
    # Specify the input and output folders
    input_folder = '/home/arth/Desktop/Grandis/rectification/BeamsAug2022_c'    # Replace with your input folder path
    output_folder = '/home/arth/Desktop/Grandis/rectification/BeamsAug2022_c_p'  # Replace with your output folder path

    process_images(input_folder, output_folder)

if __name__ == "__main__":
    main()


Could not find enough lines in image /home/arth/Desktop/Grandis/rectification/BeamsAug2022_c/U19 (1).jpg
Failed to process image /home/arth/Desktop/Grandis/rectification/BeamsAug2022_c/U19 (1).jpg
Could not find enough lines in image /home/arth/Desktop/Grandis/rectification/BeamsAug2022_c/U41 (2).jpg
Failed to process image /home/arth/Desktop/Grandis/rectification/BeamsAug2022_c/U41 (2).jpg
Could not find enough lines in image /home/arth/Desktop/Grandis/rectification/BeamsAug2022_c/U41 (1).jpg
Failed to process image /home/arth/Desktop/Grandis/rectification/BeamsAug2022_c/U41 (1).jpg
Could not find enough lines in image /home/arth/Desktop/Grandis/rectification/BeamsAug2022_c/U30 (2).jpg
Failed to process image /home/arth/Desktop/Grandis/rectification/BeamsAug2022_c/U30 (2).jpg
Could not find enough lines in image /home/arth/Desktop/Grandis/rectification/BeamsAug2022_c/U35 (2).jpg
Failed to process image /home/arth/Desktop/Grandis/rectification/BeamsAug2022_c/U35 (2).jpg
Could not find 

The issue arises because the previous method relies heavily on detecting straight edges using hough line transform, which may not work well if the edges of the beam are not perfectly straight due to instance segmentation. additionally, if the black background occupies a small area compared to the beam, edge detection might not produce reliable results.

Alternative solution:

# 2. Contour detection and polygon approximation:

Contour detection:

1. detect the contours of the object (beam) in the image.
2. since the beam occupies most of the image, the largest contour should correspond to the beam.

Contour approximation:

1. approximate the detected contour to a polygon with fewer vertices using `cv2.approxPolyDP`.
2. we aim to approximate the beam's contour to a quadrilateral.

Perspective correction:

1. if we obtain a quadrilateral, we can use its corner points to compute the perspective transformation.
2. this will "flatten" the perspective, making both sides of the beam occupy the entire height of the image.

**Assumptions**

- Beam occupies most of the iamge
- The background is consistent(this will help in thresholding and contour detection)

In [14]:
import os
import cv2
import numpy as np

def order_points(pts):
    # Initialize a list of coordinates that will be ordered
    # in the following order: top-left, top-right, bottom-right, bottom-left
    rect = np.zeros((4, 2), dtype="float32")

    # Sum and difference of points to find corners
    s = pts.sum(axis=1)
    diff = np.diff(pts, axis=1)

    # Top-left point has the smallest sum, bottom-right has the largest sum
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]

    # Top-right point has the smallest difference, bottom-left has the largest difference
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]

    return rect

def correct_perspective(image_path):
    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error loading image {image_path}")
        return None

    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Thresholding to create a binary image
    _, thresh = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)

    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        print(f"No contours found in image {image_path}")
        return None

    # Assume the largest contour is the beam
    largest_contour = max(contours, key=cv2.contourArea)

    # Approximate the contour to a polygon
    peri = cv2.arcLength(largest_contour, True)
    epsilon = 0.02 * peri  # Adjust this value if necessary
    approx = cv2.approxPolyDP(largest_contour, epsilon, True)

    if len(approx) != 4:
        print(f"Could not approximate contour to quadrilateral in image {image_path}")
        return None

    # Get the corner points
    pts = approx.reshape(4, 2)
    rect = order_points(pts)

    # Compute width and height of the new image
    (tl, tr, br, bl) = rect

    width_top = np.linalg.norm(tr - tl)
    width_bottom = np.linalg.norm(br - bl)
    max_width = max(int(width_top), int(width_bottom))

    height_left = np.linalg.norm(bl - tl)
    height_right = np.linalg.norm(br - tr)
    max_height = max(int(height_left), int(height_right))

    # Destination points
    dst = np.array([
        [0, 0],
        [max_width - 1, 0],
        [max_width - 1, max_height - 1],
        [0, max_height - 1]], dtype="float32")

    # Compute the perspective transform matrix and apply it
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (max_width, max_height))

    return warped

def process_images(input_folder, output_folder):
    # Create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Get list of image files in the input folder
    image_files = [f for f in os.listdir(input_folder)
                   if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

    if not image_files:
        print(f"No images found in {input_folder}")
        return

    for image_file in image_files:
        image_path = os.path.join(input_folder, image_file)
        result = correct_perspective(image_path)

        if result is not None:
            # Save the result in the output folder
            output_path = os.path.join(output_folder, image_file)
            cv2.imwrite(output_path, result)
            print(f"Processed and saved: {output_path}")
        else:
            print(f"Failed to process image {image_path}")

def main():
    # Specify the input and output folders
    input_folder = '/home/arth/Desktop/Grandis/rectification/uknown_c'    # Replace with your input folder path
    output_folder = '/home/arth/Desktop/Grandis/rectification/uknown_c_p'  # Replace with your output folder path

    process_images(input_folder, output_folder)

if __name__ == "__main__":
    main()


Processed and saved: /home/arth/Desktop/Grandis/rectification/uknown_c_p/cropped_7699-4-4-_JPG.rf.55ef0ce98309ac50e3aacb301b77fecf.jpg
Processed and saved: /home/arth/Desktop/Grandis/rectification/uknown_c_p/cropped_7725-3-1-_JPG.rf.1d0de76d379b46db7ec2180715a8bbcc.jpg
Processed and saved: /home/arth/Desktop/Grandis/rectification/uknown_c_p/cropped_7753-3-4-_JPG.rf.a29bec127d647d9569878bae6500789c.jpg
Processed and saved: /home/arth/Desktop/Grandis/rectification/uknown_c_p/cropped_7688-2-1-_JPG.rf.c2ee39cf5025202ec45970b86b35ab00.jpg
Processed and saved: /home/arth/Desktop/Grandis/rectification/uknown_c_p/cropped_7663-5-2-_JPG.rf.05cbb772c50961db0d33bd29013863be.jpg
Processed and saved: /home/arth/Desktop/Grandis/rectification/uknown_c_p/cropped_7581-1-2-_JPG.rf.0a4ab53002770ea1703887fb3ea5e660.jpg
Processed and saved: /home/arth/Desktop/Grandis/rectification/uknown_c_p/cropped_7710-8-2-_JPG.rf.52aeaee75203bbef0063175f7c121ca6.jpg
Processed and saved: /home/arth/Desktop/Grandis/rectifi

In [15]:
import os

def compare_directories(dir1, dir2):
    # List files in both directories
    files_dir1 = sorted(os.listdir(dir1))
    files_dir2 = sorted(os.listdir(dir2))

    # Determine the maximum length to iterate over
    max_len = max(len(files_dir1), len(files_dir2))

    # Pad the shorter list with None to avoid index errors
    files_dir1.extend([None] * (max_len - len(files_dir1)))
    files_dir2.extend([None] * (max_len - len(files_dir2)))

    # Compare files and output messages
    for i in range(max_len):
        file1 = files_dir1[i]
        file2 = files_dir2[i]

        if file1 == file2:
            print(f"Pair {i + 1}: '{file1}' and '{file2}' are the same.")
        else:
            print(f"Pair {i + 1}: '{file1}' and '{file2}' are different.")

def main():
    # Specify the directories to compare
    dir1 = '/home/arth/Desktop/Grandis/rectification/Grandis Images JMAJ 2_c'  # Replace with the path to the first directory
    dir2 = '/home/arth/Desktop/Grandis/rectification/Grandis Images JMAJ 3_c'  # Replace with the path to the second directory

    # Check if directories exist
    if not os.path.isdir(dir1):
        print(f"Directory '{dir1}' does not exist.")
        return
    if not os.path.isdir(dir2):
        print(f"Directory '{dir2}' does not exist.")
        return

    compare_directories(dir1, dir2)

if __name__ == "__main__":
    main()


Pair 1: 'cropped_7003-1-1-_JPG.rf.d5fb921129e11b002133a5cb619f3ebd.jpg' and 'cropped_7003-1-1-_JPG.rf.d5fb921129e11b002133a5cb619f3ebd.jpg' are the same.
Pair 2: 'cropped_7003-1-2-_JPG.rf.7ee4e39cd51d52c72eee002a0451e29b.jpg' and 'cropped_7003-1-2-_JPG.rf.7ee4e39cd51d52c72eee002a0451e29b.jpg' are the same.
Pair 3: 'cropped_7003-1-3-_JPG.rf.853a0a09a3d18fc2d53b93270a7ebb03.jpg' and 'cropped_7003-1-3-_JPG.rf.853a0a09a3d18fc2d53b93270a7ebb03.jpg' are the same.
Pair 4: 'cropped_7003-1-4-_JPG.rf.25cfccc64532d48becdf7f1d8b1f3c92.jpg' and 'cropped_7003-1-4-_JPG.rf.25cfccc64532d48becdf7f1d8b1f3c92.jpg' are the same.
Pair 5: 'cropped_7003-2-1-_JPG.rf.1b594e11e6d23ef1905b38ad23609a01.jpg' and 'cropped_7003-2-1-_JPG.rf.1b594e11e6d23ef1905b38ad23609a01.jpg' are the same.
Pair 6: 'cropped_7003-2-2-_JPG.rf.6b6d58823eb19258ccf9e0449e04d157.jpg' and 'cropped_7003-2-2-_JPG.rf.6b6d58823eb19258ccf9e0449e04d157.jpg' are the same.
Pair 7: 'cropped_7003-2-3-_JPG.rf.8b7732a166bd7381f9310d91542b3bdf.jpg' and 